# Life Expectancy Analysis & Modeling Using WHO & UN Data

## Initial Data Analysis

To:&nbsp;&nbsp;&nbsp;&nbsp; [Magnimind](https://magnimindacademy.com/)

From: Matt Curcio, matt.curcio.us@gmail.com

Date: 2023-01-29

Re:&nbsp;&nbsp;&nbsp; Initial Data Analysis

---

## Summary

## TO DO
add where the data is from? 
What website?
Any background

---


- The orginal dataset, `Life_Expectancy_Data.csv`, displayed **193 unique nations**.  

- However, **10 countries had only one entry each**. 
  - Cook Islands, Dominica, Marshall Islands, Monaco, Nauru, Niue, Palau, Saint Kitts and Nevis, San Marino, Tuvalu
  - These 10 countries had ONLY the year = 2013.  

- Therefore the cleaned dataset and analysis has **183 usable countries**.


---


Purpose: This notebook investigates *missing values, imputing any missing values by column mean*.

Input: `Life_Expectancy_Data.csv`

Output: `Clean_LE_Data_w_Means_1.csv`

In [1]:
# Common Python Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Load data

- Rename columns for clarity

In [2]:
path = '../data/raw/'
filename = 'Life_Expectancy_Data.csv'

column_names = ['Country','Year','Status','LifeExpectancy','AdultMort',
                'InfD','EtOH','PercExpen','HepB','Measles',
                'BMI','lt5yD','Polio','TotalExpen','DTP','HIV',
                'GDP','Population','Thin1_19y','Thin5_9y','Income',
                'Education']

df = pd.read_csv(path+filename, names=column_names, header=0)
df.shape

(2938, 22)

## Check for null values

In [3]:
print('='*38)
print('Shape:',df.shape)
print('='*38)
print('Column              No. missing values')
print('='*38)

df.isnull().sum()

Shape: (2938, 22)
Column              No. missing values


Country             0
Year                0
Status              0
LifeExpectancy     10
AdultMort          10
InfD                0
EtOH              194
PercExpen           0
HepB              553
Measles             0
BMI                34
lt5yD               0
Polio              19
TotalExpen        226
DTP                19
HIV                 0
GDP               448
Population        652
Thin1_19y          34
Thin5_9y           34
Income            167
Education         163
dtype: int64

### NOTE 1: 
- The feature 'LifeExpectancy' has 10 missing values. 
 
- Therefore the 10 rows that have NAN values will be deleted. Because 'LifeExpectancy' is a Dependent variable, I will delete those 10 observations with NO labels rather than impute them.

- Drop 10 rows containing null in `LifeExpectancy` column
 
- The 'LifeExpectancy' feature appear to be **Missing Completely at Random(MCAR)**. The main advantage of **MCAR** is that the analysis is unbiased. Data lost with design fault do not impact other parameters in the model. 

## Exactly what countries have missing Life Expectancy data.column_names?

In [4]:
# Exactly what countries have missing Life Expectancy data.column_names?

left_out = df.loc[df['LifeExpectancy'].isnull()]

print(left_out['Country'])

624              Cook Islands
769                  Dominica
1650         Marshall Islands
1715                   Monaco
1812                    Nauru
1909                     Niue
1958                    Palau
2167    Saint Kitts and Nevis
2216               San Marino
2713                   Tuvalu
Name: Country, dtype: object


In [5]:
left_out

,Country,Year,Status,LifeExpectancy,AdultMort,InfD,EtOH,PercExpen,HepB,Measles,...,Polio,TotalExpen,DTP,HIV,GDP,Population,Thin1_19y,Thin5_9y,Income,Education
624,Cook Islands,2013,Developing,NaN,NaN,0,0.01,0.000000,98.0,0,...,98.0,3.58,98.0,0.1,NaN,NaN,0.1,0.1,NaN,NaN
769,Dominica,2013,Developing,NaN,NaN,0,0.01,11.419555,96.0,0,...,96.0,5.58,96.0,0.1,722.756650,NaN,2.7,2.6,0.721,12.7
1650,Marshall Islands,2013,Developing,NaN,NaN,0,0.01,871.878317,8.0,0,...,79.0,17.24,79.0,0.1,3617.752354,NaN,0.1,0.1,NaN,0.0
1715,Monaco,2013,Developing,NaN,NaN,0,0.01,0.000000,99.0,0,...,99.0,4.30,99.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN
1812,Nauru,2013,Developing,NaN,NaN,0,0.01,15.606596,87.0,0,...,87.0,4.65,87.0,0.1,136.183210,NaN,0.1,0.1,NaN,9.6
1909,Niue,2013,Developing,NaN,NaN,0,0.01,0.000000,99.0,0,...,99.0,7.20,99.0,0.1,NaN,NaN,0.1,0.1,NaN,NaN
1958,Palau,2013,Developing,NaN,NaN,0,NaN,344.690631,99.0,0,...,99.0,9.27,99.0,0.1,1932.122370,292.0,0.1,0.1,0.779,14.2
2167,Saint Kitts and Nevis,2013,Developing,NaN,NaN,0,8.54,0.000000,97.0,0,...,96.0,6.14,96.0,0.1,NaN,NaN,3.7,3.6,0.749,13.4
2216,San Marino,2013,Developing,NaN,NaN,0,0.01,0.000000,69.0,0,...,69.0,6.50,69.0,0.1,NaN,NaN,NaN,NaN,NaN,15.1
2713,Tuvalu,2013,Developing,NaN,NaN,0,0.01,78.281203,9.0,0,...,9.0,16.61,9.0,0.1,3542.135890,1819.0,0.2,0.1,NaN,0.0


## 10 countries have only one entry each, for example, 'Cook Island'

In [6]:
df.loc[df['Country'] == 'Cook Islands']

,Country,Year,Status,LifeExpectancy,AdultMort,InfD,EtOH,PercExpen,HepB,Measles,...,Polio,TotalExpen,DTP,HIV,GDP,Population,Thin1_19y,Thin5_9y,Income,Education
624,Cook Islands,2013,Developing,NaN,NaN,0,0.01,0.0,98.0,0,...,98.0,3.58,98.0,0.1,NaN,NaN,0.1,0.1,NaN,NaN


### NOTE 2:
 
- I will not impute data for the missing 10 nations. 
- Therefore it is reasonable to delete the ten countries which make up ten rows.  

In [7]:
df.dropna(subset=['LifeExpectancy'], inplace=True) # 10 rows deleted

In [8]:
print('='*38)
print('Shape:',df.shape)
print('='*38)

Shape: (2928, 22)


### NOTE 3: 
- The three features with the highest percent of mising vlaues are:

|  | Feature | Number Missing | % Missing |
|--|:--------|---------------:|----------:|
|1 | Population | 644/2928 | 22.0% |
|2 | HepB | 553/2928 |  18.9% |
|3 | GDP | 448/2928 | 15.3% |

### NOTE 4: 
- Drop feature columns ['Population', 'HepB', 'GDP'] where % Missing is greater than 15%.


- More data scraping or gathering needs to be done in at least 5 areas.
   - 1 	Country Population
   - 2 	Hepititus B Vaccination rates
   - 3 	Gross Domestic Product
   - 4 	Total Expenditure of Country Funds: Health Related
   - 5 	Ethanol Comsumption per capita

In [9]:
df.drop(['Population', 'HepB', 'GDP'], axis=1, inplace=True)

## Imputation using column means

In [10]:
df['InfD'].fillna(np.mean(df.InfD), inplace=True)
df['EtOH'].fillna(np.mean(df.EtOH), inplace=True)
df['PercExpen'].fillna(np.mean(df.PercExpen), inplace=True)
df['Measles'].fillna(np.mean(df.Measles), inplace=True)
df['BMI'].fillna(np.mean(df.BMI), inplace=True)
df['Polio'].fillna(np.mean(df.Polio), inplace=True)
df['TotalExpen'].fillna(np.mean(df.TotalExpen), inplace=True)
df['DTP'].fillna(np.mean(df.DTP), inplace=True)
df['Thin1_19y'].fillna(np.mean(df.Thin1_19y), inplace=True)
df['Thin5_9y'].fillna(np.mean(df.Thin5_9y), inplace=True)
df['Income'].fillna(np.mean(df.Income), inplace=True)
df['Education'].fillna(np.mean(df.Education), inplace=True)

## Re-Check null data points

In [11]:
print('\nShape of Cleaned and Imputed dataframe:', df.shape)
      
df.isnull().sum()


Shape of Cleaned and Imputed dataframe: (2928, 19)


Country           0
Year              0
Status            0
LifeExpectancy    0
AdultMort         0
InfD              0
EtOH              0
PercExpen         0
Measles           0
BMI               0
lt5yD             0
Polio             0
TotalExpen        0
DTP               0
HIV               0
Thin1_19y         0
Thin5_9y          0
Income            0
Education         0
dtype: int64

## Save intermediate dataframe

In [12]:
path = '../data/processed/'
fileName = 'Clean_LE_Data_w_Means_1.csv'

df.to_csv(path+fileName, index=False)